# Cali Pfleger
## 05/25/2024
- Model iCESM and CESM Last Millenium Ensemble 
- Climate System: Pacific Walker Circulation 
- Index: Sea Level Pressure Index (hPa)
- Variable: d18Op (0/00) (per mille)
- Time period: 1850-2005 
- Forcings: Full, GHG, LULC, OZA

# Calculations:
- Index Time Series (trend line, p-value, standered error (+1/-1) ribbon and ensmeble average)
- Climatological Average Maps
- Global Average Time Series (standered error ribbon and ensmeble average)
- Linear Trend Maps (95% CI significance stippling)- 
- Correlation with Index Maps (95% CI significance stippling)- 
- Effect of Forcing Maps (Correlation - PI Control Run Corrleation)
GCM Changes in Vertical Velocity (ω) Across the Tropical Pacific.
iCAM5 = 2xCO2 Experiment minus Modern Control, Meridional Mean (−20 to 20 latitude).Change in vertical velocity (ω) in the experiment minus the control in color filled contour plot. Black contour lines show control simulation. Sign convention is negative = upward motion, positive = downward motion. Thus, red colors in Western Pacific indicate a reduction in the magnitude of the negative values in the control compared to the experiment. Blue colors indicate less positive values in the experiment compared to the control, or less downward motion.

- Vertical Profile Maps (dD(per mille), omega, temperature (C), pressure (hPa))
- 

In [1]:
# Set Parameters and Naming 
model ='Model iCESM and CESM '
region = 'PWC Index ' # 'Eastern Pacific ' 'Western Pacific ' 
variable = 'SLP' #Sea Level Pressure
period = "Post Industrial"# "Full LME" #"Pre Industrial",
timep = "_post"# "_pre" # 
var_name = ['Full', 'GHG', 'LULC','OZA']#, 'ANTHRO']
timep_title = "Post Industrial"#'Pre Industrial' 

In [2]:
# Index Box Parameters 
## Make geom for E and W PWC index boxes
from shapely import geometry
from shapely.geometry.polygon import LinearRing, Polygon
Elats = [-5, -5, 5, 5]
Elons = [90, 150, 150,90]
Ering = LinearRing(list(zip(Elons, Elats)))
Wlats = [-5, -5, 5, 5]
Wlons = [-150,-90,-90,-150]
Wring = LinearRing(list(zip(Wlons, Wlats)))

# Read in Files 

In [3]:
## Set Paths

In [4]:
pwd

'/glade/u/home/calipfleger/stats/PWC_Paper/pwc_clean'

In [5]:
basedir_atm = '/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/'
file_prefix= '.cam.h0.' 
i_model = '/b.ie12.B1850C5CN.f19_g16.LME.' ## read in 2&3 iLME Ensembles 
i_model1 = '/b.ie12.B1850CN.f19_g16.' ## read in iLME ens 1
model = '/b.e11.BLMTRC5CN.f19_g16.'

# Import Packages

In [6]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import scipy.stats as stats
import xarray as xr
from tqdm import tqdm
import os
from pprint import pprint
#import nctoolkit
# colors for lines (color blind friendly colors: https://gist.github.com/thriveth/8560036)
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

import matplotlib as mpl  # # Plotting
import matplotlib.pyplot as plt  # python plotting package
import matplotlib.colors as mcolors
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from pandas.plotting import register_matplotlib_converters
from colorspacious import cspace_converter
mpl.rcParams['figure.dpi'] = 300

from collections import namedtuple
from collections import OrderedDict
cmaps = OrderedDict()

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from colorspacious import cspace_converter

In [7]:
import xarray as xr
import numpy as np
import cftime
import pandas as pd
from numpy import *
from scipy import stats
from netCDF4 import Dataset as nc
from netCDF4 import num2date
import netCDF4
import datetime
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt #python plotting package
from matplotlib import cm
import matplotlib.colors as mcolors
from colorspacious import cspace_converter
from collections import OrderedDict
cmaps = OrderedDict()
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import nctoolkit
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cpf
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely import geometry
from collections import namedtuple
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon

# Calculations and Functions 

In [8]:
# CESM converting month 01 to January 
from itertools import product
from cftime import DatetimeNoLeap
year = np.linspace(0,1872, 1872) 
dates = [DatetimeNoLeap(year, month, 1) for year, month in product(range(1850, 2006), range(1, 13))]
da = xr.DataArray(np.arange(1872), coords=[dates], dims=['time'], name='time')

In [9]:
## Set control year 
i_cntlyear = np.linspace(0,2400, 2400) 
i_cntldates = [DatetimeNoLeap(i_cntlyear, month, 1) for i_cntlyear, month in product(range(650, 850), range(1, 13))]
i_cntlda =xr.DataArray(np.arange(2400), coords=[i_cntldates], dims=['time'], name='time')

In [10]:
# define grid cell weighting to acount for grid cells in extratropics 
def weights(w):
    coslat = np.cos(np.deg2rad(w.lat)) #  Take the cosine of latitude (first converting to radians)
    weight_factor = coslat / coslat.mean(dim='lat')#  And divide by its mean value
    computed_weight = w * weight_factor # .mean(dim=('time', 'lon', 'lat'))
    return computed_weight

In [11]:
## Calculate Ensemble average 
def ensav(e):
    ens_avg = e.mean(dim = 'ensemble')
    return ens_avg

In [12]:
# Code Dr. Samantha Stevenson UCSB
#mask_lon = (mytos.lon >= regbox[2]) & (mytos.lon <= regbox[3])
#mask_lat = (mytos.lat >= regbox[0]) & (mytos.lat <= regbox[1])
#hsst=mytos.where(mask_lon & mask_lat, drop=True).squeeze().isel(lev=23).isel(lev=23).isel(lev=23)

In [13]:
# PWC Index
def compute_index(c):
    east = c.sel(lat=slice(-5,5), lon=slice(210,270))
    west = c.sel(lat=slice(-5,5), lon=slice(90,150))
    #east = c.sel(lat=slice(-10,10), lon=slice(190,250))
    #west = c.sel(lat=slice(-10,10), lon=slice(90,150))
    avg_east = east.mean(('lat', 'lon'))
    avg_west =  west.mean(('lat', 'lon'))
    clim_index = (avg_east- avg_west).compute()
    return clim_index

In [14]:
# Function for meridional mean and lat lon range 
def mer(m):
    #m_we = weights(m)
    mer_lat= m.sel(lat=slice(-20, 20)).mean(dim='lat')
    mer_lon = mer_lat.sel(lon=slice(110, 270))
    mer_clim = clim(mer_lon)
    mer_time = mer_clim.mean(dim='time')
    mer_avg = mer_time#.mean(dim = 'ensemble')
    return mer_avg

In [15]:
## Calculate Anomaly relative to Climatology for variable 
def clim(tc): 
    #units = tc/100 #convert d18O 
    #convert precip 1000*24*3600
    c = tc.assign_coords({'time':(da.time)}) #convert time dimestion to have 01 be january 
    clim_var = c.groupby('time.month').mean('time') #calculate monthly climatology
    clim_anom = c.groupby('time.month') - clim_var # calculate the anomaly relative to the climatology 
    #anom_weights = weights(clim_anom) #weight functino to account for lat grid size 
    return  clim_anom #return the anomaly calcuated by the anomaly with time and lat conversion 

In [16]:
## Calculate Anomaly relative to Climatology for variable 
def clim_cntl(tc): 
    #units = tc/100 #convert d18O 
    #convert precip 1000*24*3600
    c = tc.assign_coords({'time':(i_cntlda.time)}) #convert time dimestion to have 01 be january 
    clim_var = c.groupby('time.month').mean('time') #calculate monthly climatology
    clim_anom = c.groupby('time.month') - clim_var # calculate the anomaly relative to the climatology 
    #anom_weights = weights(clim_anom) #weight functino to account for lat grid size 
    return clim_anom #return the anomaly calcuated by the anomaly with time and lat conversion 

In [17]:
## Calculate Ensemble average 
def clim_avg(ca):
    #convert precip 1000*24*3600
    c = ca.assign_coords({'time':(da.time)})
    time_avg = c.mean('time')
    ens_avg = time_avg.mean(dim = 'ensemble')
    return ens_avg

In [18]:
# Calculate Correlation Coefficent using xr.corr 
def cor(g,t):
    ds= []
    fcor =xr.corr(g,t, dim = 'time') # g is the global data set and t is the time series index 
    ds.append(fcor) # add correlation coeefiecent to dataset 
    cords= xr.concat(ds, dim='ensemble')  # create ensmenble dimesion for correlation coeeficent 
    cor_avg = cords.mean('ensemble') # calculate ensemble average for correlation coeeficent 
    return cor_avg 

In [19]:
def l_trend(var,lon,lat,time,sig):
    nlon=len(lon)
    nlat=len(lat)
    nt=len(time)
    vart=zeros(nlat*nlon)
    varp=zeros(nlat*nlon)
    if len(var.shape)== 3:        
        var1=reshape(var,(nt,nlat*nlon)) 
        for i in range(nlat*nlon):
            v=var1[:,i]  
            vart[i], intercept, r_value, varp[i], std_err=stats.linregress(time,v) # slope, intercept, r, p, std_error
        vart=reshape(vart,(nlat,nlon))
        varp=reshape(varp,(nlat,nlon))  
        vt.append(vart)
        vp.append(varp)
        slope = np.stack(vt, axis=0)#, dim='ensemble')
        pvalue = np.stack(vp, axis=0)#, dim='ensemble')
        sig_pval = ma.masked_greater(pvalue, sig)
        pval_avg = avg(sig_pval)
        slope_avg = avg(slope)
        return (slope, sig_pval, slope_avg, pval_avg) 

In [20]:
print('Index Files: ' +  model + period ,''+ region )

Index Files: /b.e11.BLMTRC5CN.f19_g16.Post Industrial PWC Index 


# Read in Index Files: TS

In [21]:
var = 'OMEGA' #set output variable name for reading in file 

In [22]:
%%time
ens = '' #Full forcing ensemble 
file_suffix= '.185001-200512.nc'
ds = []# initialise array:
for member in tqdm(range(1,2)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model1+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
    
for member in tqdm(range(2,3+1)): #loop to read in the 1-9 ensemble members 
    id = str(member)
    file = id+file_prefix+var+ file_suffix # set file path 
    member = xr.open_dataset(basedir_atm + var+ i_model +ens +'00' +file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)

for member in tqdm(range(1,9+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
    
for member in tqdm(range(10,13+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ model+ens +'0'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
    
var_full=  xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)})#/100)  

100%|██████████| 4/4 [00:14<00:00,  3.59s/it]


CPU times: user 3min 17s, sys: 11.2 s, total: 3min 29s
Wall time: 4min 11s


In [23]:
%%time
ens = 'GHG.'  # GHG ensemble 
file_suffix= '.185001-200512.nc'
ds = [] # initialise array:
for member in tqdm(range(1,2)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
for member in tqdm(range(3,4+1)): #add in 4th ensmble when permission is not denied 
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ model +ens +'00' +file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
var_ghg =  xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)})#/100)  

100%|██████████| 3/3 [00:05<00:00,  1.87s/it]


CPU times: user 1min 44s, sys: 26.7 s, total: 2min 10s
Wall time: 2min 25s


In [24]:
%%time
ens = 'O3AER.'
file_suffix= '.185001-200512.nc'
ds = []
for member in (1,3,4,5):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm + var+ i_model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)

ens = 'OZONE_AER.'
for member in tqdm(range(1,5+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ model+ ens + '00'+file).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)

var_oza= xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)})#/100)  

100%|██████████| 5/5 [00:14<00:00,  2.96s/it]


CPU times: user 1min 50s, sys: 6.2 s, total: 1min 56s
Wall time: 2min 15s


In [27]:
%%time
ens = 'LULC.'  #Lulc ensemble 
file_suffix= '.169001-200512.nc'
ds = []
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ i_model+ ens + '00'+file).sel(time = slice('1850-02-01', '2006-01-01')).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
ens = 'LULC_HurttPongratz.'
file_suffix= '.185001-200512.nc'
for member in tqdm(range(1,3+1)):
    id = str(member)
    file = id+file_prefix+var+ file_suffix
    member = xr.open_dataset(basedir_atm+ var+ model+ ens + '00'+file).sel(time = slice('1850-02-01', '2006-01-01')).sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()
    ds.append(member)
var_lulc = xr.concat(ds, dim='ensemble').sel(time = slice('1850-02-01', '2006-01-01')).assign_coords({'time':(da.time)})#/100)  

 33%|███▎      | 1/3 [00:00<00:01,  1.69it/s]

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


100%|██████████| 3/3 [00:07<00:00,  2.60s/it]


CPU times: user 1min 14s, sys: 4.02 s, total: 1min 18s
Wall time: 1min 31s


In [28]:
icntl_index = xr.open_dataset('/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/OMEGA/b.ie12.B1850CN.f19_g16.850cntl.001.cam.h0.OMEGA.065001-084912.nc').sel(lat=slice(-35,35),lon=slice(75,290)).OMEGA.squeeze()

In [33]:
variable = 'OMEGA18'
var_full.isel(lev =18).to_netcdf(variable+'full.nc')
var_ghg.isel(lev =18).to_netcdf(variable+'ghg.nc')
var_oza.isel(lev =18).to_netcdf(variable+'oza.nc')
var_lulc.isel(lev =18).to_netcdf(variable+'lulc.nc')
icntl_index.isel(lev =18).to_netcdf(variable+'cntl.nc')